In [1]:
from erddapy import ERDDAP
import pandas as pd
import urllib.request

# help(ERDDAP.get_search_url)

In [ ]:
# reformat this:
# search for any data set within a small time and space window
# get info for each - 
# check variable names for those key words
# download that dataset

In [2]:
# example of setting time/space search criteria


# kw = {
#     "min_lon": -72.0,
#     "max_lon": -69.0,
#     "min_lat": 3f8.0,
#     "max_lat": 41.0,
#     "min_time": "2016-07-10T00:00:00Z",
#     "max_time": "2017-02-10T00:00:00Z",
# }
# search_url = e.get_search_url(response="html", **kw)

In [3]:
search_keywrds = ['par','photon','radiance','downwelling','turbidity','sediment','ssc','cdom','chlorophyll']

min_time = "2024-01-01T00:00:00Z"
max_time = "2024-01-02T00:00:00Z"
kw = {
    "min_lon": -170.0,
    "max_lon": -43.0,
    "min_lat": 2.0,
    "max_lat": 80.0,
    "min_time": min_time,
    "max_time": max_time,
}


In [13]:
help(e.get_search_url)

Help on method get_search_url in module erddapy.erddapy:

get_search_url(response: Optional[str] = None, search_for: Optional[str] = None, protocol: Optional[str] = None, items_per_page: int = 1000, page: int = 1, **kwargs) -> str method of erddapy.erddapy.ERDDAP instance
    Build the search URL for the `server` endpoint provided.
    
    Args:
        search_for: "Google-like" search of the datasets' metadata.
    
            - Type the words you want to search for, with spaces between the words.
                ERDDAP will search for the words separately, not as a phrase.
            - To search for a phrase, put double quotes around the phrase
                (for example, `"wind speed"`).
            - To exclude datasets with a specific word, use `-excludedWord`.
            - To exclude datasets with a specific phrase, use `-"excluded phrase"`
            - Searches are not case-sensitive.
            - You can search for any part of a word. For example,
                search

In [4]:
# try searching for all the variables at just one erddap server
e = ERDDAP(
    server="http://www.neracoos.org/erddap/", 
    protocol="tabledap", 
    response="csv")


allsites = list()
for val in search_keywrds:
    print('Searching ' + e.server + ' for ' + val)
    url = e.get_search_url(search_for=val, response="csv", **kw)
    try:
        temp = pd.read_csv(url)["Dataset ID"].unique()
    except:
        print("----no matching sites exist")
    else:
        allsites.extend(temp)

FF = pd.DataFrame(allsites,columns=['sites'])
FF = FF['sites'].unique()


Searching http://www.neracoos.org/erddapfor photon
----no matching sites exist
Searching http://www.neracoos.org/erddapfor radiance
----no matching sites exist
Searching http://www.neracoos.org/erddapfor downwelling
----no matching sites exist
Searching http://www.neracoos.org/erddapfor turbidity
36
Searching http://www.neracoos.org/erddapfor sediment
----no matching sites exist
Searching http://www.neracoos.org/erddapfor ssc
----no matching sites exist
Searching http://www.neracoos.org/erddapfor cdom
1
Searching http://www.neracoos.org/erddapfor chlorophyll
36


In [5]:
print(FF)

['A01_optics_s_all' 'E01_sbe37_all' 'F01_sbe37_all' 'M01_suna_all'
 'A01_sbe37_all' 'B01_sbe37_all' 'A01_doppler_rt' 'I01_sbe37_all'
 'M01_sbe37_all' 'M01_accelerometer_all' 'E01_aanderaa_all'
 'B01_doppler_rt' 'M01_aanderaa_all' 'A01_sbe16_disox_all'
 'A01_aanderaa_o2_all' 'A01_optode_all' 'A01_met_all' 'F01_met_all'
 'M01_met_all' 'I01_met_all' 'B01_met_all' 'A01_waves_mstrain_all'
 'F01_waves_mstrain_all' 'E01_waves_mstrain_all' 'M01_waves_mstrain_all'
 'I01_waves_mstrain_all' 'B01_waves_mstrain_all' 'E01_met_all'
 'F01_aanderaa_all' 'F01_accelerometer_all' 'B01_aanderaa_all'
 'B01_accelerometer_all' 'I01_accelerometer_all' 'E01_accelerometer_all'
 'I01_aanderaa_all' 'A01_accelerometer_all']


In [11]:
e.dataset_id = FF[0]
e.constraints = {
    "time>=": min_time,
    "time<=": max_time}

url = e.get_download_url()

df = pd.read_csv(url,parse_dates=True)
print(df.columns)

Index(['station', 'mooring_site_desc', 'time', 'chlorophyll', 'chlorophyll_qc',
       'turbidity', 'turbidity_qc', 'longitude', 'latitude', 'depth'],
      dtype='object')


In [7]:
# pd.set_option('display.max_rows', None)
# info_url = e.get_info_url(dataset_id=FF[0])
# df = pd.read_csv(info_url)
# print(df)
# print((df.columns))